## Fruit Classification Using Computer Vision: A Study with the Fruits-360 Dataset

**Goal: Train a CV model on the Fruits-360 dataset to classify different types of fruits based on images**

### Roadmap

1) Follow steps in README.md

2) Preprocess images from dataset

3) Train CNN Model with a optimizer and loss function. 

4) Evaluate Performance 

5) Save trained model for deployement used for inference in your app!

7) Use results and Nutrients API to get nutrtion into your user profile in your app! (Part 2 of project)

In [16]:
import torch
import os

import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support



## Step 2 - Preprocess images

In [13]:
class FruitsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_to_index = {}  # Map labels to indices

        # Load image paths and labels
        for idx, label in enumerate(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            if os.path.isdir(label_dir):
                self.label_to_index[label] = idx  # Assign an index to each label
                for image_name in os.listdir(label_dir):
                    self.image_paths.append(os.path.join(label_dir, image_name))
                    self.labels.append(idx)  # Store the index instead of the label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)  # Convert label to a tensor

# Define transformations
transform = transforms.Compose([
    transforms.Resize((100, 100)),  # Resize images to 100x100
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Create dataset and dataloader
dataset = FruitsDataset(root_dir='/Users/maryjojohnson/Documents/mdst-w24/fruits-360_dataset_100x100/fruits-360/Training', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


## Step 3 - Train CNN model 

In [14]:

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 25 * 25, 128)  # Adjust based on image size
        self.fc2 = nn.Linear(128, len(set(dataset.labels)))  # Adjust based on number of classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 25 * 25)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

print('Finished Training')

Epoch [1/10], Loss: 0.2917
Epoch [2/10], Loss: 0.0432
Epoch [3/10], Loss: 0.0236
Epoch [4/10], Loss: 0.0195
Epoch [5/10], Loss: 0.0150
Epoch [6/10], Loss: 0.0103
Epoch [7/10], Loss: 0.0154
Epoch [8/10], Loss: 0.0129
Epoch [9/10], Loss: 0.0104
Epoch [10/10], Loss: 0.0169
Finished Training


## Step 4: Evaluate Performance

In [17]:

# Assuming you have a test dataset directory similar to the training dataset
test_dataset = FruitsDataset(root_dir='/Users/maryjojohnson/Documents/mdst-w24/fruits-360_dataset_100x100/fruits-360/Test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to evaluate the model
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in dataloader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)  # Get the index of the max log-probability
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    # Calculate accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    # Calculate precision, recall, and F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

# Evaluate the model
evaluate_model(model, test_loader)

Accuracy: 0.9555
Precision: 0.9610
Recall: 0.9555
F1 Score: 0.9544


## Step 5: Save Trained model 

In [18]:
# Save the model's state dictionary
torch.save(model.state_dict(), 'simple_cnn_model.pth')